# Preprocess Data

In [1]:
import pandas
import numpy

In [2]:
books = pandas.read_csv('archive/Books.csv')
# books.to_pickle('books')
ratings = pandas.read_csv('archive/Ratings.csv')

C:\Users\stanl\AppData\Local\Temp\ipykernel_12684\4152650104.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pandas.read_csv('archive/Books.csv')


In [3]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [4]:
ratings = ratings[ratings['ISBN'].isin(books['ISBN'])] # Some books aren't in the books database but are in ratings
ratings = ratings[ratings['Book-Rating'] != 0]
# ratings.to_pickle('ratings')
ratings.head()

,User-ID,ISBN,Book-Rating
1,276726,0155061224,5
3,276729,052165615X,3
4,276729,0521795028,6
8,276744,038550120X,7
16,276747,0060517794,9


In [5]:
user_ratings_count = ratings.groupby('User-ID').count()
print(user_ratings_count.describe())
user_ratings_count = user_ratings_count[user_ratings_count['ISBN'] >= 3] # Users with atleast than 5 ratings
user_ratings_count.shape

               ISBN   Book-Rating
count  68091.000000  68091.000000
mean       5.637191      5.637191
std       41.742511     41.742511
min        1.000000      1.000000
25%        1.000000      1.000000
50%        1.000000      1.000000
75%        3.000000      3.000000
max     6943.000000   6943.000000


(20194, 2)

In [6]:
book_ratings_count = ratings.groupby('ISBN').count()
print(book_ratings_count['Book-Rating'].quantile(0.99))
book_ratings_count = book_ratings_count[book_ratings_count['User-ID'] >= 24] #.sort_values(by='User-ID', ascending=False) # Books with atleast than 100 ratings
# book_ratings_count.head()
# book_ratings_count[book_ratings_count.index == '8804342838']
# book_ratings_count['Book-Rating'].quantile(numpy.arange(0.9, 1, 0.01))

24.0


In [7]:
book_ratings = ratings.groupby('ISBN').mean()
book_ratings

,User-ID,Book-Rating
ISBN,,
0000913154,171118.0,8.0
0001046438,23902.0,9.0
000104687X,23902.0,6.0
0001047213,23902.0,9.0
0001047973,17923.0,9.0
...,...,...
B0001FZGPI,222207.0,7.0
B0001FZGRQ,192093.0,9.0
B0001GMSV2,101983.0,8.0


In [8]:
# ratings = ratings[ratings['User-ID'].isin(user_ratings_count.index)] # Remove users with less than 5 ratings, Perhaps this is not a good idea
ratings = ratings[ratings['ISBN'].isin(book_ratings_count.index)] # Remove books with less than 20 ratings

# Replace 0 ratings with the average
ratings = ratings[ratings['Book-Rating'] != 0]
# avg_ratings = ratings[non_zero_ratings].groupby('ISBN')['Book-Rating'].mean().round(0)
# ratings.loc[~non_zero_ratings, 'Book-Rating'] = ratings['ISBN'].map(avg_ratings)
# ratings['Book-Rating'] = 1
# ratings.head()

In [9]:
matrix = ratings.pivot(index='ISBN', columns='User-ID', values='Book-Rating').fillna(0)
# matrix.to_pickle('matrix')
matrix[matrix.index == '0395177111']

User-ID,9,16,26,32,39,42,51,56,75,91,...,278800,278807,278828,278832,278836,278843,278844,278846,278851,278854
ISBN,,,,,,,,,,,,,,,,,,,,,


# Find Nearest Neighbors

In [10]:
from sklearn.neighbors import NearestNeighbors
import joblib

model = NearestNeighbors(metric='cosine', n_neighbors=7, algorithm='brute')
model.fit(matrix)
# joblib.dump(model, 'model.pkl')

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=7)

In [11]:
book = matrix[matrix.index == '0395177111'].values
book

array([], shape=(0, 28915), dtype=float64)

In [12]:
_, neigh_ind = model.kneighbors(book)

ValueError: Found array with 0 sample(s) (shape=(0, 28915)) while a minimum of 1 is required by NearestNeighbors.

In [ ]:
neigh_ind

In [ ]:
for i in neigh_ind[0]:
    print(books[books['ISBN'] == matrix.index[i]])